In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import re
import selenium
from pandas import DataFrame

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": ChromeDriverManager().install()}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
browser = init_browser()
url = "https://www.fragrantica.com/search/"
browser.visit(url)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/jenniferdean/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [11]:
html = browser.html
soup = bs(html, "html.parser")

In [ ]:
for i in range(1920,2021)
    browser.find_by_css('input[type="number"]')[0].fill(2020)
    browser.find_by_css('input[type="number"]')[1].fill(2020)

In [37]:
for i in range(0,34):
    browser.find_by_css('button[class="button"]').click()

In [42]:
html = browser.html
soup = bs(html, "html.parser")

In [6]:
block1 = soup.find("span", class_="grid-x grid-margin-x grid-margin-y small-up-3 medium-up-2 large-up-4 perfumes-row text-center")

In [9]:
#COLLECT PERFUME NAME

perfume_name_list = []

for perf_text in block1.find_all("a", href=True):
    perf_name = perf_text.get_text()
    print(perf_name.replace('\n', '').strip(" "))
    perfume_name.append(perf_name.replace('\n', '').strip(" "))

Knize Ten
Les Exclusifs de Chanel Bel Respiro
Eau de Fleurs de Cedrat
Lily of the Valley
Ellada
Lysval
Eva
Novaya Zarya
Sweet Pea Душистый горошек
Eau perlee
Eau de Cologne L'Originale
Eau givree
Eau nacree
Eau irisee
Eau epicee
Eau boisee
Mury
Lavande
Jeux
Heliotrope
Cuir de Russie
Lacdor


In [10]:
#COLLECT COMPANY NAME
perfume_company_list = []

for company_text in block1.find_all("p"):
    if company_text.small != None:  
        print (company_text.small.get_text())
        perfume_company.append(company_text.small.get_text())

Knize
Chanel
Guerlain
Yardley
Severnoye Siyanie
Girard
Lubin
Nouvelle Etoile – Новая Заря
Nouvelle Etoile – Новая Заря
Jardin de France
Mont St Michel
Jardin de France
Jardin de France
Jardin de France
Jardin de France
Jardin de France
Mury Paris
Mury Paris
Mury Paris
Mury Paris
Mury Paris
Lubin


In [24]:
for i in range(len(perfume_name)):
    browser.find_by_css('span[class="link-span"]')[i].click()
    
    html = browser.html
    soup = bs(html, "html.parser")
    
    perfume_name = perfume_name_list[i]
    perfume_comp = perfume_company_list[i]
    
    for_gender = soup.find("small").get_text()
    
    perfume_image = soup.find_all("div", class_="cell small-12")[1].find("img")["src"]
    
    rating = float(soup.find("p", class_="info-note").find_all("span")[0].get_text())
    
    number_votes = int(soup.find("p", class_="info-note").find_all("span")[2].get_text().replace(',', ''))

    description = soup.find_all("div", class_="cell small-12")[3].get_text()

    main_accords = soup.find_all("div", class_="cell accord-box")
        for j in main_accords:
            accord_name = main_accords[j].get_text()
            accord_value = float(main_accords[j].find("div", class_="accord-bar")["style"].rsplit("width: ")[1].strip("%;"))
            
    notes = soup.find_all("div", attrs={"style": "display: flex; justify-content: center; text-align: center; flex-flow: row wrap; align-items: flex-end; padding: 0.5rem;"})
    #Top Notes
    print(notes[0].find_all("div")[0].get_text())
    print(notes[0].find_all("div")[3].get_text())

    #Middle Notes
    print(notes[1].find_all("div")[0].get_text())
    print(notes[1].find_all("div")[3].get_text())

    #Base Notes
    print(notes[2].find_all("div")[0].get_text())
    print(notes[2].find_all("div")[3].get_text())

    

'Maison Francis Kurkdjian'

In [55]:
len(perfume_name)

1000

In [48]:
browser.find_by_css('input[type="number"]')[0].fill(2020)
browser.find_by_css('input[type="number"]')[1].fill(2020)

In [53]:
for i in range(0,34):
    browser.find_by_css('button[class="button"]').click()

In [ ]:
browser.find_by_css('button[class="button"]').click()

In [ ]:
#Clicking into the individual perfumes

browser.find_by_css('span[class="link-span"]')[0].click()

In [ ]:
html = browser.html
soup = bs(html, "html.parser")

In [ ]:
for_gender = soup.find("small").get_text()
for_gender

In [ ]:
perfume_image = soup.find_all("div", class_="cell small-12")[1].find("img")["src"]
perfume_image

In [ ]:
#All Accords

main_accords = soup.find_all("div", class_="cell accord-box")
main_accords

In [ ]:
accord_name = main_accords[1].get_text()
accord_name

In [ ]:
accord_value = float(main_accords[1].find("div", class_="accord-bar")["style"].rsplit("width: ")[1].strip("%;"))
accord_value

In [ ]:
rating = float(soup.find("p", class_="info-note").find_all("span")[0].get_text())
rating

In [ ]:
number_votes = int(soup.find("p", class_="info-note").find_all("span")[2].get_text().replace(',', ''))
number_votes

In [ ]:
description = soup.find_all("div", class_="cell small-12")[3].get_text()
description

In [12]:
#All Notes

notes = soup.find_all("div", attrs={"style": "display: flex; justify-content: center; text-align: center; flex-flow: row wrap; align-items: flex-end; padding: 0.5rem;"})

#Top Notes
print(notes[0].find_all("div")[0].get_text())
print(notes[0].find_all("div")[3].get_text())

#Middle Notes
print(notes[1].find_all("div")[0].get_text())
print(notes[1].find_all("div")[3].get_text())

#Base Notes
print(notes[2].find_all("div")[0].get_text())
print(notes[2].find_all("div")[3].get_text())


Geranium
Rosemary
Carnation
Patchouli
Leather
Castoreum


In [41]:
for i in range(len(notes)):
    print(notes[i].find_all("div")[5].get_text())

Rosemary
Patchouli
Castoreum


In [69]:
notes

[<div style="display: flex; justify-content: center; text-align: center; flex-flow: row wrap; align-items: flex-end; padding: 0.5rem;"><div style="margin: 0.2rem; display: flex; justify-content: center; flex-direction: column; text-align: center; opacity: 0.881966; position: relative;"><div><img loading="lazy" src="https://fimgs.net/mdimg/sastojci/t.21.jpg" style="width: 3.2rem;"/></div><div><a href="https://www.fragrantica.com/notes/Geranium-21.html"><span class="link-span"></span></a>Geranium</div></div><div style="margin: 0.2rem; display: flex; justify-content: center; flex-direction: column; text-align: center; opacity: 0.840458; position: relative;"><div><img loading="lazy" src="https://fimgs.net/mdimg/sastojci/t.49.jpg" style="width: 2.975rem;"/></div><div><a href="https://www.fragrantica.com/notes/Rosemary-49.html"><span class="link-span"></span></a>Rosemary</div></div><div style="margin: 0.2rem; display: flex; justify-content: center; flex-direction: column; text-align: center;

In [83]:
len(notes[0].find_all("span", class_="link-span"))

6

In [84]:
len(notes[1].find_all("span", class_="link-span"))

8

In [85]:
len(notes[2].find_all("span", class_="link-span"))

6

In [59]:
j = 0

for i in range(len(notes)):
    try:
        print(notes[i].find_all("div")[j].get_text())
        print(notes[i].find_all("div")[j+3].get_text())
        print(notes[i].find_all("div")[j+6].get_text())
        print(notes[i].find_all("div")[j+9].get_text())
        print(notes[i].find_all("div")[j+12].get_text())
        print(notes[i].find_all("div")[j+15].get_text())
        print(notes[i].find_all("div")[j+18].get_text())
        print(notes[i].find_all("div")[j+21].get_text())
        print(notes[i].find_all("div")[j+23].get_text())
        print(notes[i].find_all("div")[j+25].get_text())
    except IndexError:
        print(f" {i} is Out of Index Range")

Geranium
Rosemary
Bergamot
Orange
Petitgrain
Amalfi Lemon
 0 is Out of Index Range
Carnation
Patchouli
Sandalwood
Orris
tincture of rose
Virginia Cedar
Cinnamon
Orange Blossom
Orange Blossom
 1 is Out of Index Range
Leather
Castoreum
oak moss
Amber
Musk
Vanilla
 2 is Out of Index Range


In [98]:
#for i in range(len(notes[0].find_all("span", class_="link-span"))):
    print(notes[1].find_all("div")[2].get_text())
    print(notes[1].find_all("div")[5].get_text())
    print(notes[1].find_all("div")[8].get_text())
    print(notes[1].find_all("div")[11].get_text())
    print(notes[1].find_all("div")[14].get_text())
    print(notes[1].find_all("div")[17].get_text())
    print(notes[1].find_all("div")[20].get_text())
    print(notes[1].find_all("div")[23].get_text())

Carnation
Patchouli
Sandalwood
Orris
tincture of rose
Virginia Cedar
Cinnamon
Orange Blossom


In [99]:
i = 2

for i in range(len(notes[1].find_all("span", class_="link-span"))):
    print(notes[1].find_all("div")[i].get_text())
    i += 3
    
#     i = 0 // i = 2
#     i = 1 // i = 4
#     i = 2 // i = 


Carnation

Carnation
Patchouli

Patchouli
Sandalwood



In [ ]:
#all voting chart numbers

voting = soup.find_all("div", class_="cell small-1 medium-1 large-1")


#Longevity

long_v_weak = int(voting[0].get_text())
long_weak = int(voting[1].get_text())
long_moderate = int(voting[2].get_text())
long_long_last = int(voting[3].get_text())
long_eternal = int(voting[4].get_text())

#Sillage

sill_intimate = int(voting[5].get_text())
sill_moderate = int(voting[6].get_text())
sill_strong = int(voting[7].get_text())
sill_enormus = int(voting[8].get_text())

#Gender

gender_female = int(voting[9].get_text())
gender_more_fem = int(voting[10].get_text())
gender_unisex = int(voting[11].get_text())
gender_more_male = int(voting[12].get_text())
gender_male = int(voting[13].get_text())

#Price Value
value_w_over = int(voting[14].get_text())
value_over = int(voting[15].get_text())
value_ok = int(voting[16].get_text())
value_good = int(voting[17].get_text())
value_great = int(voting[18].get_text())

In [ ]:
browser.back()

In [ ]:
voting